In [1]:
!pip install tensorflowjs --no-cache-dir

In [1]:
!pip install bertopic
!pip install sentence-transformers
!pip install gensim
!pip install nltk
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import Library

In [3]:
import nltk
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import re

from bertopic import BERTopic
from gensim.models import Word2Vec
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm

/usr/local/lib/python3.12/dist-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


#Download Package tambahan NLTK

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

#Read Data


In [5]:
data1 = pd.read_csv('/content/drive/MyDrive/nana/dataset_tiktok-comments-scraper_2025-08-30_14-48-28-486.csv')
data2 = pd.read_csv('/content/drive/MyDrive/nana/dataset_tiktok-comments-scraper_2025-08-30_14-53-31-785.csv')

In [6]:
data = pd.concat([data1, data2])

#Cleaning & EDA

In [7]:
data.duplicated().sum()

1419

In [8]:
data.shape

(7965, 9)

In [9]:
data = data.drop_duplicates()

In [10]:
data.duplicated().sum()

0

In [11]:
data.shape

(6546, 9)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6546 entries, 0 to 3998
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   text               6539 non-null   object 
 1   diggCount          6546 non-null   int64  
 2   replyCommentTotal  5310 non-null   float64
 3   createTimeISO      6546 non-null   object 
 4   uniqueId           6546 non-null   object 
 5   videoWebUrl        6546 non-null   object 
 6   uid                6546 non-null   int64  
 7   cid                6546 non-null   int64  
 8   avatarThumbnail    6546 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 511.4+ KB


In [13]:
data.head(10)

,text,diggCount,replyCommentTotal,createTimeISO,uniqueId,videoWebUrl,uid,cid,avatarThumbnail
0,the real presiden OMON OMON,14,0.0,2025-04-16T23:39:05.000Z,tempoyakudang17,https://www.tiktok.com/@partaigerindra/video/7...,7315285121450804229,7494059573180121864,https://p16-common-sign-sg.tiktokcdn-us.com/to...
1,spam like ah,0,0.0,2025-06-08T00:08:18.000Z,ngaprrr,https://www.tiktok.com/@partaigerindra/video/7...,7509803052002640903,7513363593908749057,https://p16-common-sign-sg.tiktokcdn-us.com/to...
2,#FREEPALESTINE🇵🇸🇵🇸🇵🇸\n#SAVEPALESTINE🇵🇸🇵🇸,0,0.0,2025-07-28T09:05:37.000Z,bayy_max1,https://www.tiktok.com/@partaigerindra/video/7...,7170304377276957722,7532056373811561223,https://p16-common-sign-va.tiktokcdn-us.com/to...
3,free palistin 🇵🇸🇵🇸🇵🇸🇵🇸,0,0.0,2025-06-15T15:15:36.000Z,fazz1085,https://www.tiktok.com/@partaigerindra/video/7...,7443787704447403025,7516195047097221906,https://p16-common-sign-sg.tiktokcdn-us.com/to...
4,TOLONG TANGKAP BOIKOT TAMAN SAFARI!!!,1114,31.0,2025-04-19T07:33:57.000Z,use4u_,https://www.tiktok.com/@partaigerindra/video/7...,7354328076853036038,7494924183861560082,https://p16-common-sign-sg.tiktokcdn-us.com/to...
5,manis di awal😹,211,3.0,2025-04-13T12:06:39.000Z,m.qalibil_qiramm,https://www.tiktok.com/@partaigerindra/video/7...,7314590367838913542,7492767963389674248,https://p16-common-sign-sg.tiktokcdn-us.com/to...
6,"KRITIKNYA DIBALESS ATUHHH, SEKELAS PARTAI PRES...",170,3.0,2025-04-11T14:49:55.000Z,aufdhirga_,https://www.tiktok.com/@partaigerindra/video/7...,6977948582885704705,7492067868629697288,https://p16-common-sign-va.tiktokcdn-us.com/to...
7,kasian Ama koruptor hah ??,209,4.0,2025-04-13T12:15:27.000Z,kaguradevil,https://www.tiktok.com/@partaigerindra/video/7...,6731020826583598082,7492770196899300114,https://p19-common-sign-useast2a.tiktokcdn-us....
8,pak tolong selidiki orientasl circus Indonesia...,102,1.0,2025-04-20T15:29:56.000Z,nimas3364,https://www.tiktok.com/@partaigerindra/video/7...,7479469529317721095,7495417941178401554,https://p16-common-sign-va.tiktokcdn-us.com/mu...
9,kok admin gerindra kini udah gak kayak dulu la...,1568,60.0,2025-04-11T10:12:56.000Z,putoijoo0,https://www.tiktok.com/@partaigerindra/video/7...,6943169261369361410,7491996453553931029,https://p16-common-sign-sg.tiktokcdn-us.com/to...


In [14]:
data.isnull().sum()

,0
text,7
diggCount,0
replyCommentTotal,1236
createTimeISO,0
uniqueId,0
videoWebUrl,0
uid,0
cid,0
avatarThumbnail,0


In [15]:
data.dropna(inplace=True)

In [16]:
data.shape

(5303, 9)

In [17]:
data.columns

Index(['text', 'diggCount', 'replyCommentTotal', 'createTimeISO', 'uniqueId',
       'videoWebUrl', 'uid', 'cid', 'avatarThumbnail'],
      dtype='object')

In [18]:
data.drop(columns=['diggCount', 'replyCommentTotal', 'createTimeISO', 'uniqueId',
       'videoWebUrl', 'uid', 'cid', 'avatarThumbnail'], axis=1, inplace=True)

In [19]:
def preprocessText(text:str):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = re.sub(r'[^\w\s#@/:%\.\-]', '', text)
  words = word_tokenize(text)
  stopWords = set(stopwords.words('indonesian'))
  words = [word for word in words if word not in stopWords]
  return " ".join(words)

In [20]:
data['afterCleaning'] = data['text'].apply(preprocessText)

In [21]:
data.head(10)

,text,afterCleaning
0,the real presiden OMON OMON,the real presiden omon omon
1,spam like ah,spam like ah
2,#FREEPALESTINE🇵🇸🇵🇸🇵🇸\n#SAVEPALESTINE🇵🇸🇵🇸,freepalestine savepalestine
3,free palistin 🇵🇸🇵🇸🇵🇸🇵🇸,free palistin
4,TOLONG TANGKAP BOIKOT TAMAN SAFARI!!!,tolong tangkap boikot taman safari
5,manis di awal😹,manis
6,"KRITIKNYA DIBALESS ATUHHH, SEKELAS PARTAI PRES...",kritiknya dibaless atuhhh sekelas partai presi...
7,kasian Ama koruptor hah ??,kasian ama koruptor hah
8,pak tolong selidiki orientasl circus Indonesia...,tolong selidiki orientasl circus indonesia per...
9,kok admin gerindra kini udah gak kayak dulu la...,admin gerindra udah gak kayak lagii


In [22]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

uniqueWords = set()
data['afterCleaning'].str.split().apply(uniqueWords.update)

stemmerWords = {word: stemmer.stem(word) for word in tqdm(uniqueWords)}

100%|██████████| 7119/7119 [10:06<00:00, 11.74it/s]


tessssssssssssss



In [23]:
stemming = lambda text: ' '.join([stemmerWords[word] for word in text.split()])
data['afterStemming'] = data['afterCleaning'].apply(stemming)

In [24]:
data.head(10)

,text,afterCleaning,afterStemming
0,the real presiden OMON OMON,the real presiden omon omon,the real presiden omon omon
1,spam like ah,spam like ah,spam like ah
2,#FREEPALESTINE🇵🇸🇵🇸🇵🇸\n#SAVEPALESTINE🇵🇸🇵🇸,freepalestine savepalestine,freepalestine savepalestine
3,free palistin 🇵🇸🇵🇸🇵🇸🇵🇸,free palistin,free palistin
4,TOLONG TANGKAP BOIKOT TAMAN SAFARI!!!,tolong tangkap boikot taman safari,tolong tangkap boikot taman safari
5,manis di awal😹,manis,manis
6,"KRITIKNYA DIBALESS ATUHHH, SEKELAS PARTAI PRES...",kritiknya dibaless atuhhh sekelas partai presi...,kritik dibaless atuhhh kelas partai presiden d...
7,kasian Ama koruptor hah ??,kasian ama koruptor hah,kasi ama koruptor hah
8,pak tolong selidiki orientasl circus Indonesia...,tolong selidiki orientasl circus indonesia per...,tolong selidik orientasl circus indonesia buda...
9,kok admin gerindra kini udah gak kayak dulu la...,admin gerindra udah gak kayak lagii,admin gerindra udah gak kayak lagi


tesss

#Preprocessing

In [25]:
text = data['afterStemming'].tolist()

In [26]:
textTokenized = [text.split() for text in text]

In [27]:
word2vecModel = Word2Vec(sentences=textTokenized, vector_size=100, window=5, min_count=1, workers=4)

In [28]:
textTokenized

[['the', 'real', 'presiden', 'omon', 'omon'],
 ['spam', 'like', 'ah'],
 ['freepalestine', 'savepalestine'],
 ['free', 'palistin'],
 ['tolong', 'tangkap', 'boikot', 'taman', 'safari'],
 ['manis'],
 ['kritik',
  'dibaless',
  'atuhhh',
  'kelas',
  'partai',
  'presiden',
  'diem',
  'aja'],
 ['kasi', 'ama', 'koruptor', 'hah'],
 ['tolong',
  'selidik',
  'orientasl',
  'circus',
  'indonesia',
  'budak',
  'bos',
  'kerja',
  'budak',
  'isolasi',
  'dunia'],
 ['admin', 'gerindra', 'udah', 'gak', 'kayak', 'lagi'],
 ['negara',
  'ambang',
  'krisis',
  'jd',
  'jgn',
  'paksa',
  'klu',
  'anggaranny',
  'gk',
  'lapang',
  'kerja',
  'utk',
  'ortunya',
  'lbh',
  'desak',
  'dri',
  'pd',
  'mkn',
  'gratis',
  'utk',
  'penuh',
  'isi',
  'septi',
  'teng'],
 ['tolong', 'notice', 'keras', 'taman', 'safari'],
 ['sumpah',
  'tukang',
  'kunci',
  'amanah',
  'banding',
  'sumpah',
  'jabat',
  'kitab',
  'suci'],
 ['gimana',
  'nih',
  'gerindrapa',
  'wowo',
  'kasihan',
  'korupsikecew

In [29]:
docEmbedding = []

for doc in textTokenized:
  wordVectors = [word2vecModel.wv[word] for word in doc if word in word2vecModel.wv]
  if wordVectors:
    docVector = np.mean(wordVectors, axis=0)
  else:
    docVector = np.zeros(word2vecModel.vector_size)
  docEmbedding.append(docVector)

In [30]:
docEmbedding

[array([-0.3093397 ,  0.10408926,  0.23434663,  0.32335013,  0.17312726,
        -0.22152114,  0.14386313,  0.68819135, -0.19148086, -0.2309129 ,
        -0.03072816, -0.62672555, -0.15548909,  0.2662838 ,  0.18402754,
        -0.10947026,  0.41813388,  0.27514493, -0.43897304, -0.6403946 ,
         0.06694151,  0.00183239,  0.32385117, -0.43035507,  0.44738454,
        -0.08784073, -0.24000087,  0.27733767, -0.30110973,  0.06431564,
         0.3028915 , -0.15815395,  0.10561781, -0.415762  , -0.24480855,
         0.135371  ,  0.32717252, -0.05353227,  0.2656092 , -0.05151835,
        -0.15429458,  0.10748508, -0.44618526, -0.13476656,  0.0538015 ,
         0.13024089, -0.03189681,  0.1342848 ,  0.03051313,  0.32566896,
         0.05289049, -0.38661498, -0.15478525, -0.09347355, -0.05233354,
         0.0415488 ,  0.3729043 , -0.09971936, -0.32099372,  0.27426556,
        -0.13454205,  0.02133166,  0.11678612, -0.21430206, -0.320468  ,
         0.48854905,  0.41874152,  0.1544766 , -0.2

#Build Model Bertopic

In [31]:
bertSentenceModel =  SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
docEmbeddingBert = bertSentenceModel.encode(text, show_progress_bar=True)

Batches:   0%|          | 0/166 [00:00<?, ?it/s]

In [33]:
combinedEmbeddings = np.hstack((docEmbedding, docEmbeddingBert))

In [34]:
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

indonesian_stopwords = stopwords.words('indonesian')

vectorizer = CountVectorizer(
    ngram_range=(1, 3),
    min_df=10,
    max_df=0.95,
    stop_words=indonesian_stopwords
)

umap_model = UMAP(
    n_neighbors=30,
    n_components=3,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)

model = BERTopic(
    language='indonesian',
    calculate_probabilities=True,
    verbose=True,
    umap_model=umap_model,
    vectorizer_model=vectorizer
)

In [35]:
topics, probs = model.fit_transform(text, embeddings=combinedEmbeddings)

2025-10-12 06:37:56,308 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-12 06:38:41,877 - BERTopic - Dimensionality - Completed ✓
2025-10-12 06:38:41,878 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-12 06:38:45,142 - BERTopic - Cluster - Completed ✓
2025-10-12 06:38:45,148 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-12 06:38:45,414 - BERTopic - Representation - Completed ✓


tessssssssssssssss

#Evaluasi & Intrepertasi Hasil

In [36]:
topicDict = model.get_topics()

if isinstance(topicDict, dict) :
  # Extract topic words as a list of lists
  topicWords = [[word for word, _ in model.get_topic(topic_id)] for topic_id in topicDict.keys()]

  # Filter out empty documents from textTokenized
  cleanedTextTokenized = [text for text in textTokenized if text]

  # Create dictionary from the cleaned textTokenized
  dictionary = corpora.Dictionary(cleanedTextTokenized)

  # Filter out words from topicWords that are not in the dictionary
  cleanedTopicWords = []
  for topic in topicWords:
      cleaned_topic = [word for word in topic if word in dictionary.token2id]
      if cleaned_topic: # Only include non-empty cleaned topics
          cleanedTopicWords.append(cleaned_topic)

  if cleanedTopicWords: # Only calculate coherence if there are valid topics
      coherencemodel = CoherenceModel(topics=cleanedTopicWords, texts=cleanedTextTokenized, dictionary=dictionary, coherence='c_v')
      coherenceScore = coherencemodel.get_coherence()
      print(coherenceScore)
  else:
      print("No valid topics left after cleaning.")

0.6215575469020328


In [37]:
model.get_topic_freq()

,Topic,Count
2,-1,1456
20,0,482
25,1,209
7,2,140
48,3,134
...,...,...
15,98,11
92,99,11
80,100,11
99,101,11


In [38]:
model.get_topic(0)

[('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

In [39]:
model.visualize_barchart(top_n_topics=10)

In [40]:
model.visualize_hierarchy()

In [41]:
model.visualize_topics()

In [42]:
model.visualize_term_rank()

Analisis Topik:

**Topik 1:**
  Kata Kunci: nyesel, pilih, prabowo, milih, kecewa, gw, mending
  
  Interpretasi: Topik ini jelas mencerminkan rasa penyesalan dan kekecewaan pengguna terkait pilihan mereka pada Prabowo.

**Topik 2:**
  Kata Kunci: gerindra, tutup, turun, prabowo
  
  Interpretasi: Topik ini kemungkinan besar membahas isu terkait 'Gerindra', mungkin mengenai kinerja atau isu spesifik seperti 'dapur mbg' yang disebutkan di topik lain, atau hal-hal yang 'turun'.

**Topik 3:**
  Kata Kunci: saverajaampat
  
  Interpretasi: Kata kunci 'saverajaampat' berulang menunjukkan fokus yang kuat pada isu lingkungan atau konservasi terkait Raja Ampat. Komentar ini bisa berupa seruan untuk menyelamatkan wilayah tersebut.

**Topik 4:**
  Kata Kunci: bayar, mbg, blm, tuh, duit, gimana, dpt
  
  Interpretasi: Topik ini tampaknya berpusat pada masalah keuangan, kemungkinan besar terkait dengan 'mbg' (makan bergizi gratis?) dan pertanyaan mengenai 'bayar' atau 'duit' yang belum jelas statusnya.

**Topik 5:**
  Kata Kunci: save raja ampat, save raja, save, raja ampat, ampat, raja, ampat save raja
  
  Interpretasi: Mirip dengan Topik 3, topik ini kembali menekankan 'Save Raja Ampat', mungkin dengan variasi frasa yang berbeda, menunjukkan kepedulian yang meluas terhadap isu ini.

**Topik 6:**
  Kata Kunci: makan, gratis, makan gratis, sekolah, dapet, ku, gizi
  
  Interpretasi: Topik ini secara spesifik membahas program 'makan gratis', kemungkinan besar terkait dengan sekolah atau cara mendapatkannya ('dapet').

**Topik 7:**
  Kata Kunci: negara, rakyat, maju, indonesia, korupsi, kerja, aja
  
  Interpretasi: Topik ini memiliki cakupan yang lebih luas, menyentuh isu-isu negara, rakyat, kemajuan, dan korupsi. Ini bisa jadi komentar-komentar umum tentang kondisi negara atau harapan terhadap kepemimpinan.

**Topik 8:**
  Kata Kunci: ampat save raja, raja ampat save, save raja ampat, ampat save, save raja, save, raja ampat
  
  Interpretasi: Frasa berulang 'ampat save raja' dan variasinya kembali menegaskan fokus pada isu Raja Ampat, menunjukkan bahwa ini adalah isu yang signifikan dalam komentar.

**Topik 9:**
  Kata Kunci: tolong, min, min tolong, uang, bayar, kasi, orang
  
  Interpretasi: Kata kunci 'kasian', 'koruptor', dan 'hah' mungkin mencerminkan komentar yang bernada sindiran atau keprihatinan terhadap koruptor atau situasi terkait korupsi.

**Topik 10:**
  Kata Kunci: gerindra, admin, prabowo, lihat, sppi, ga, mohon
  
  Interpretasi: 'Taman safari', 'tolong', 'tindak', 'boikot', dan 'selamat' sangat mengindikasikan diskusi mengenai isu atau insiden di Taman Safari, kemungkinan besar yang memerlukan tindakan atau boikot.